<div style='background-image: url("header.png") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">Tutorial by Mondaic</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">For Salvus version 0.11.25</div>
        </div>
    </div>
</div>

# 1D Models

## 1D model file format

The 1D model file format is adopted from AxiSEM and also used as a standard within the NASA Insight mission. It is based on `KEY VALUE` pairs as follows:

##### KEY VALUE definitions

The key value format can accomodate 3 different types of values: single values, list of values and tables,
all separated by white space. Keys are case sensitive and must not have leading white space. Comments
are indicated byt the # character. Continuation lines of tables are indented by at least one blank.

```
# single value
KEY VALUE

# list of values
KEY VALUE1 VALUE2 VALUE3

# table
KEY HEADER1 HEADER2
  VALUE11 VALUE21
  VALUE12 VALUE22
```

#### Available Keys for Salvus

For Salvus, the following keys can be used:

`NAME` Name of the model. type: string, default: filename without ending.

`DESCRIPTION` Reference or verbal description of the model.

`ANISOTROPIC` Whether the model is anisotropic. type: boolean, default: `false`.

`ANELASTIC` Whether the model includes an attenuation model. type: boolean, default: `false`.

`REFERENCE_FREQUENCY` Frequency at which the seismic velocities are defined in Hz. type: float, default:
`1.0`.

`UNITS` Units used for density, seismic velocities and depth/radius, either `m` for SI units (m, m/s, kg/m 3 )
or `km` for (km, km/s, g/cm 3 ). type: string, default: `m`.

`COLUMNS` Table containing the depth dependent model parameters. type: float.

#### Depth dependent parameters

Discontinuities in the model (both first and second order) are detected based on repeated radius/depth values. In between these discontinuities, paramteres are assumed to be smooth and interpolated using splines (cubic by default). The table columns in the table `COLUMNS` have no particular order, the rows can be sorted either from top to bottom or vice versa. The table needs to contain at least these colums:

`radius` or `depth`

`rho`

`vp` and `vs` if ANISOTROPIC is false.

`vpv`, `vph`, `vsv`, `vsh` and `eta` if `ANISOTROPIC` is `true`.

`QMU` and `QKAPPA` if `ANELASTIC` is `true`


#### Sample File 1: a local isotropic elastic model with 3 layers

A three layer 1D model where the seismic velocities have gradients in the upper 2 layers and constant below.

```
NAME         true_model
UNITS        m
COLUMNS      depth rho vp vs
    0.0     2384.4 3500.0 2020.0
    2000.0  2441.9 3850.0 2223.0
    2000.0  2570.1 4725.0 2728.0
    7000.0  2780.8 6475.0 3738.0
    7000.0  2835.5 7000.0 4041.0
    10000.0 2835.5 7000.0 4041.0
```

#### Sample File 2: a global PREM model

some lines removed as indicated by `[...]`:

```
# Input file for Salvus
NAME         prem_ani
ANELASTIC       T
ANISOTROPIC     T
UNITS        m
COLUMNS       radius      rho      vpv      vsv      qka      qmu      vph      vsh      eta
            6371000.  2600.00  5800.00  3200.00    57827.0      600.0  5800.00  3200.00  1.00000
            6356000.  2600.00  5800.00  3200.00    57827.0      600.0  5800.00  3200.00  1.00000
#          Discontinuity   1, depth:      15.00 km < this just a comment and ignored by the software
            6356000.  2900.00  6800.00  3900.00    57827.0      600.0  6800.00  3900.00  1.00000
            6346600.  2900.00  6800.00  3900.00    57827.0      600.0  6800.00  3900.00  1.00000
#          Discontinuity   2, depth:      24.40 km
            6346600.  3380.75  8190.32  4396.02    57827.0      600.0  8190.32  4611.80  0.90039
            6335480.  3379.54  8182.26  4398.58    57827.0      600.0  8182.26  4601.82  0.90471
            [...]
            5771000.  3975.82 10157.83  5515.93    57827.0      143.0 10157.83  5515.93  1.00000
#          Discontinuity   6, depth:     600.00 km > second order discontinuity
            5771000.  3975.82 10157.76  5516.02    57827.0      143.0 10157.76  5516.02  1.00000
            [...
            3480000.  5566.46 13716.62  7264.65    57827.0      312.0 13716.62  7264.65  1.00000
#          Discontinuity  10, depth:    2891.00 km > fluid by vs=0
            3480000.  9903.44  8064.79     0.00    57827.0        0.0  8064.79     0.00  1.00000
            [...]
            1221500. 12166.33 10355.72     0.00    57827.0        0.0 10355.72     0.00  1.00000
#          Discontinuity  11, depth:    5149.50 km
            1221500. 12763.61 11028.26  3504.31     1327.7       84.6 11028.26  3504.31  1.00000
            [...]
                  0. 13088.50 11262.20  3667.80     1327.7       84.6 11262.20  3667.80  1.00000
```

## Built-in models

Salvus includes a number of popular 1D models that can directly be used for meshing.

In [ ]:
from salvus.mesh.models_1D import model

for mname in sorted(model.get_builtin_models()):
    mod = model.built_in(mname)
    print("\033[1m" + mod.name + "\033[0m")
    print(mod.description + "\n")

## Plotting 1D models

Salvus' 1D model class includes funcionality to plot 1D models in a number of different ways.

In [ ]:
# plot all model parameters
mod = model.built_in("prem_ani")
mod.plot()

In [ ]:
# plot vp vs only
mod = model.built_in("prem_iso")
mod.plot_vp_vs_profile()

In [ ]:
# compare 2 models
mod1 = model.built_in("prem_iso")
mod2 = model.built_in("ak135f")

figure = mod1.plot_vp_vs_profile(show=False)
figure = mod2.plot_vp_vs_profile(
    show=True, figure=figure, linestylemap={"VP": "--", "VS": "--"}
)

## Derived parameters

Salvus' model class can also compute quantities that derive from the 1D parameters, in particular ellipticity as a function of depth by solving Clairaut's equation and the gravity potential and force by solving Poisson's equation:

In [ ]:
mod = model.built_in("prem_ani")
mod.plot_ellipticity()

In [ ]:
mod.plot_gravity()